<a href="https://colab.research.google.com/github/ubiodee/Generate_words/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset_all.csv to dataset_all.csv


In [ ]:
import pandas as pd
data=pd.read_csv('dataset_all.csv')


In [ ]:
from sklearn.model_selection import train_test_split



# Split the dataset into training and validation sets
train_job_titles, val_job_titles = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_job_titles.reset_index(drop=True, inplace=True)
val_job_titles.reset_index(drop=True, inplace=True)

In [ ]:
Train_input=[]
Train_label=[]

for i in range(len(train_job_titles)-10000):

  title = train_job_titles.keywords[i]
  description = train_job_titles.description[i]

  Train_input.append(title)
  Train_label.append(description)


In [ ]:
Test_input=[]
Test_label=[]
for i in range(len(val_job_titles)-10000):

    title = val_job_titles.keywords[i]
    description = val_job_titles.description[i]

    Test_input.append(title)
    Test_label.append(description)

In [ ]:
import torch

# Check if a GPU is available and if not, use a CPU
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, \
    TrainingArguments, Trainer, DataCollatorWithPadding
from torch.utils.data import Dataset

## GPT-2 Small ('gpt2'): 124 million parameters.
## GPT-2 Medium ('gpt2-medium'): 345 million parameters.
## GPT-2 Large ('gpt2-large'): 774 million parameters.
## GPT-2 XL ('gpt2-xl'): 1.5 billion parameters.


# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
max_seq_length = 512

tokenized_train_inputs = tokenizer(Train_input, truncation=True, padding='max_length',max_length=max_seq_length, return_tensors="pt")
tokenized_train_outputs = tokenizer(Train_label, truncation=True, padding='max_length',max_length=max_seq_length, return_tensors="pt")

# Tokenize validation data
tokenized_val_inputs = tokenizer(Test_input, truncation=True, padding='max_length',max_length=max_seq_length, return_tensors="pt")
tokenized_val_outputs = tokenizer(Test_label, truncation=True, padding='max_length',max_length=max_seq_length, return_tensors="pt")

In [ ]:
input_ids_Train = tokenized_train_inputs.input_ids
attention_mask_Train = tokenized_train_inputs.attention_mask
labels_Train = tokenized_train_outputs.input_ids

val_input_ids = tokenized_val_inputs.input_ids
val_attention_mask = tokenized_val_inputs.attention_mask
val_labels = tokenized_val_outputs.input_ids

print(input_ids_Train.size())
print(attention_mask_Train.size())
print(labels_Train.size())

torch.Size([88669, 512])
torch.Size([88669, 512])
torch.Size([88669, 512])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
batch_size = 4
num_epochs = 10

optimizer = torch.optim.AdamW(model.parameters(),lr=0.00001,weight_decay=0.01)


val_preds = []

In [ ]:
for epoch in range(num_epochs):
    train_total_loss = 0
    val_total_loss = 0


    # Training
    model.train()
    for i in range(0, len(input_ids_Train), batch_size):
        batch_input_ids = input_ids_Train[i:i+batch_size].to(device)
        batch_attention_mask = attention_mask_Train[i:i+batch_size].to(device)
        batch_labels = labels_Train[i:i+batch_size].to(device)

        outputs = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_mask,
            labels=batch_labels
        )

        loss = outputs.loss
        train_total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        for i in range(0, len(val_input_ids), batch_size):
          batch_input_ids = val_input_ids[i:i+batch_size].to(device)
          batch_attention_mask = val_attention_mask[i:i+batch_size].to(device)
          batch_labels = val_labels[i:i+batch_size].to(device)

          outputs = model(
            input_ids=batch_input_ids,
            labels=batch_labels
          )

          # Get the predicted labels
          _, preds = torch.max(outputs.logits, dim=1)
          val_preds.extend(preds.cpu().numpy())

          loss = outputs.loss
          val_total_loss += loss.item()

    print(f"Epoch {epoch+1}, Train Loss: {train_total_loss}, Val Loss: {val_total_loss}")

    # Save the model after each epoch
    model.save_pretrained(f"model_after_epoch_{epoch}")


In [ ]:
model.train()